# Divide-and-Conquer and Sorting

Recall our three-step recipe for writing a recursive function:

 1. **Divide** the input space into at least two regions.
 2. Identify some regions as **base cases** and solve them immediately* (without recursion).
 3. Solve the remaining regions as **recursive cases**, where each makes one or more calls to the same function *but where the arguments have "gotten smaller"*.

## Merge Sort

In [9]:
def merge_sort(L, depth=0):
    """Returns a new sorted list containing the same elements as L."""
    print("."*depth, "MERGE SORT", L)
    if len(L) <= 1:
        return L
    middle = len(L) // 2
    left = L[:middle]
    right = L[middle:]
    left = merge_sort(left, depth+1)
    right = merge_sort(right, depth+1)
    output = merge(left, right)
    print("."*depth, "MERGED INTO", output)
    return output

In [3]:
def merge(left, right):
    """Assumes left and right are sorted lists.  Returns a single new
    list built, in order, from the elements of left and right."""
    output = []
    left_i = 0
    right_i = 0
    while left_i < len(left) and right_i < len(right):
        if left[left_i] < right[right_i]:
            output.append(left[left_i])
            left_i += 1
        else:
            output.append(right[right_i])
            right_i += 1
    output.extend(left[left_i:])
    output.extend(right[right_i:])
    return output

In [4]:
assert merge([1, 2], [3, 4]) == [1, 2, 3, 4]
assert merge([1, 3], [2, 4]) == [1, 2, 3, 4]

In [10]:
assert merge_sort([1, 4, 7, 3]) == [1, 3, 4, 7]

 MERGE SORT [1, 4, 7, 3]
. MERGE SORT [1, 4]
.. MERGE SORT [1]
.. MERGE SORT [4]
. MERGED INTO [1, 4]
. MERGE SORT [7, 3]
.. MERGE SORT [7]
.. MERGE SORT [3]
. MERGED INTO [3, 7]
 MERGED INTO [1, 3, 4, 7]


In [6]:
assert merge_sort([23, 3, 45, 7, 6, 11, 14, 12]) == [3, 6, 7, 11, 12, 14, 23, 45]

## In-place Merging

In [22]:
def inplace_merge_sort(L, left=0, right=None, depth=0):
    """Sorts L[left:right] in place, defaulting to all of L."""
    if right is None: right = len(L)
    print("."*depth, "MERGE SORT", L[left:right])
    if right - left <= 1:
        return L
    middle = (left + right) // 2
    # recursively sort L[left:middle]
    inplace_merge_sort(L, left, middle, depth+1)
    # recursively sort L[middle:right]
    inplace_merge_sort(L, middle, right, depth+1)
    inplace_merge(L, left, middle, right, '.'*depth)
    print("."*depth, "MERGED INTO", L[left:right])

In [2]:
def inplace_merge(L, left, middle, right):
    """Cheating function to merge sorted L[left:middle] and
    sorted L[middle:right] into sorted L[left:right]."""
    L[left:right] = merge(L[left:middle], L[middle:right])

In [4]:
L = [1, 4, 7, 3]
inplace_merge_sort(L)
assert L == [1, 3, 4, 7]

 MERGE SORT [1, 4, 7, 3]
. MERGE SORT [1, 4]
.. MERGE SORT [1]
.. MERGE SORT [4]
. MERGED INTO [1, 4]
. MERGE SORT [7, 3]
.. MERGE SORT [7]
.. MERGE SORT [3]
. MERGED INTO [3, 7]
 MERGED INTO [1, 3, 4, 7]


In [8]:
def inplace_merge(L, left, middle, right, prefix=''):
    """Merge sorted L[left:middle] and sorted L[middle:right]
    into sorted L[left:right]."""
    prefix += ':'
    print(prefix, 'MERGING', L[left:middle], L[middle:right], 'WITHIN', L)
    # base case
    if middle - left == 0 or right - middle == 0:
        return
    left_i = middle-1
    right_i = middle
    while left_i >= left and right_i < right:
        if L[left_i] < L[right_i]:
            break
        left_i -= 1
        right_i += 1
    left_i += 1
    right_i -= 1
    if left_i == left:
        while right_i < right:
            if L[left_i] < L[right_i]:
                break
            right_i += 1
        right_i -= 1
    if right_i == right-1:
        while left_i >= left:
            if L[left_i] < L[right_i]:
                break
            left_i -= 1
        left_i += 1
    big_swap(L, left_i, middle, right_i+1)
    inplace_merge(L, left, left_i, middle, prefix)
    inplace_merge(L, middle, right_i+1, right, prefix)

In [6]:
def big_swap(L, left, middle, right, prefix = ''):
    """Cheating function to swap L[left:middle] with L[middle:right]"""
    L[left:right] = L[middle:right] + L[left:middle]

In [9]:
L = [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
inplace_merge(L, 0, 6, 12)
assert L == [4, 9, 10, 15, 16, 19, 23, 26, 30, 32, 39, 47]

: MERGING [10, 15, 19, 23, 26, 32] [4, 9, 16, 30, 39, 47] WITHIN [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
:: MERGING [10, 15, 19] [4, 9, 16] WITHIN [10, 15, 19, 4, 9, 16, 23, 26, 32, 30, 39, 47]
::: MERGING [10] [4, 9] WITHIN [10, 4, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGING [] [4] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGING [9, 10] [] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
::: MERGING [15, 19] [16] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGING [15] [16] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::::: MERGING [15] [] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::::: MERGING [] [16] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
:::: MERGING [19] [] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
:: MERGING [23, 26, 32] [30, 39, 47] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::: MERGING [23, 26] [30] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 30, 32, 39, 47]
:::: MERGING [

In [14]:
def big_swap(L, left, middle, right):
    """Swap L[left:middle] with L[middle:right]"""
    if middle - left == 0 or right - middle == 0:
        return
    if middle - left <= right - middle:
        for i in range(left, middle):
            j = i + middle - left
            L[i], L[j] = L[j], L[i]
        big_swap(L, middle, middle + middle - left, right)
    else:
        for i in range(middle, right):
            j = i - (right - middle)
            L[i], L[j] = L[j], L[i]
        big_swap(L, left, middle - (right - middle), middle)

In [15]:
L = [1, 2, 3, 4]
big_swap(L, 0, 1, 4)
assert L == [2, 3, 4, 1]

In [16]:
L = [1, 2, 3, 4]
big_swap(L, 0, 2, 4)
assert L == [3, 4, 1, 2]

In [17]:
L = [1, 2, 3, 4]
big_swap(L, 0, 3, 4)
assert L == [4, 1, 2, 3]

In [18]:
# Lots of testing
for n in range(1, 10):
    for middle in range(n):
        L = list(range(n))
        big_swap(L, 0, middle, n)
        print(L, '==', list(range(middle, n)) + list(range(0, middle)))
        assert L == list(range(middle, n)) + list(range(0, middle))

[0] == [0]
[0, 1] == [0, 1]
[1, 0] == [1, 0]
[0, 1, 2] == [0, 1, 2]
[1, 2, 0] == [1, 2, 0]
[2, 0, 1] == [2, 0, 1]
[0, 1, 2, 3] == [0, 1, 2, 3]
[1, 2, 3, 0] == [1, 2, 3, 0]
[2, 3, 0, 1] == [2, 3, 0, 1]
[3, 0, 1, 2] == [3, 0, 1, 2]
[0, 1, 2, 3, 4] == [0, 1, 2, 3, 4]
[1, 2, 3, 4, 0] == [1, 2, 3, 4, 0]
[2, 3, 4, 0, 1] == [2, 3, 4, 0, 1]
[3, 4, 0, 1, 2] == [3, 4, 0, 1, 2]
[4, 0, 1, 2, 3] == [4, 0, 1, 2, 3]
[0, 1, 2, 3, 4, 5] == [0, 1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 0] == [1, 2, 3, 4, 5, 0]
[2, 3, 4, 5, 0, 1] == [2, 3, 4, 5, 0, 1]
[3, 4, 5, 0, 1, 2] == [3, 4, 5, 0, 1, 2]
[4, 5, 0, 1, 2, 3] == [4, 5, 0, 1, 2, 3]
[5, 0, 1, 2, 3, 4] == [5, 0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5, 6] == [0, 1, 2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6, 0] == [1, 2, 3, 4, 5, 6, 0]
[2, 3, 4, 5, 6, 0, 1] == [2, 3, 4, 5, 6, 0, 1]
[3, 4, 5, 6, 0, 1, 2] == [3, 4, 5, 6, 0, 1, 2]
[4, 5, 6, 0, 1, 2, 3] == [4, 5, 6, 0, 1, 2, 3]
[5, 6, 0, 1, 2, 3, 4] == [5, 6, 0, 1, 2, 3, 4]
[6, 0, 1, 2, 3, 4, 5] == [6, 0, 1, 2, 3, 4, 5]
[0, 1, 2, 3, 4, 5, 6

In [19]:
L = [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
inplace_merge(L, 0, 6, 12)
assert L == [4, 9, 10, 15, 16, 19, 23, 26, 30, 32, 39, 47]

: MERGING [10, 15, 19, 23, 26, 32] [4, 9, 16, 30, 39, 47] WITHIN [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
:: MERGING [10, 15, 19] [4, 9, 16] WITHIN [10, 15, 19, 4, 9, 16, 23, 26, 32, 30, 39, 47]
::: MERGING [10] [4, 9] WITHIN [10, 4, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGING [] [4] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGING [9, 10] [] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
::: MERGING [15, 19] [16] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGING [15] [16] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::::: MERGING [15] [] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::::: MERGING [] [16] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
:::: MERGING [19] [] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
:: MERGING [23, 26, 32] [30, 39, 47] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::: MERGING [23, 26] [30] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 30, 32, 39, 47]
:::: MERGING [

In [23]:
L = [1, 4, 7, 3]
inplace_merge_sort(L)
assert L == [1, 3, 4, 7]

 MERGE SORT [1, 4, 7, 3]
. MERGE SORT [1, 4]
.. MERGE SORT [1]
.. MERGE SORT [4]
.: MERGING [1] [4] WITHIN [1, 4, 7, 3]
.:: MERGING [1] [] WITHIN [1, 4, 7, 3]
.:: MERGING [] [4] WITHIN [1, 4, 7, 3]
. MERGED INTO [1, 4]
. MERGE SORT [7, 3]
.. MERGE SORT [7]
.. MERGE SORT [3]
.: MERGING [7] [3] WITHIN [1, 4, 7, 3]
.:: MERGING [] [3] WITHIN [1, 4, 3, 7]
.:: MERGING [7] [] WITHIN [1, 4, 3, 7]
. MERGED INTO [3, 7]
: MERGING [1, 4] [3, 7] WITHIN [1, 4, 3, 7]
:: MERGING [1] [3] WITHIN [1, 3, 4, 7]
::: MERGING [1] [] WITHIN [1, 3, 4, 7]
::: MERGING [] [3] WITHIN [1, 3, 4, 7]
:: MERGING [4] [7] WITHIN [1, 3, 4, 7]
::: MERGING [4] [] WITHIN [1, 3, 4, 7]
::: MERGING [] [7] WITHIN [1, 3, 4, 7]
 MERGED INTO [1, 3, 4, 7]


## Nuts and Bolts Puzzle

A carpenter has a whole collection of nuts and bolts of different sizes in a bag. Each nut is unique and has a corresponding unique bolt of the same size, but the disorganized carpenter has dumped them all into one bag and they are all mixed up.  Given a nut and a bolt, it's easy to try to screw them together and see whether they're perfectly sized for each other, or the bolt is too big (won't screw in), or the bolt is too big (turns loosely).  However, the carpenter's eyes aren't very good, so they can't reliably compare two nuts to each other or two bolts to each other.  How can we best match up the nuts and bolts?

<img src="img/nuts_and_bolts.png">

## Quicksort

In [1]:
def quicksort(L, left = 0, right = None):
    """Sorts L[left:right] in place, defaulting to all of L."""
    if right is None: right = len(L)
    if right - left <= 1: return
    middle = partition(L, left, right)
    quicksort(L, left, middle)
    quicksort(L, middle+1, right)
    #done

In [2]:
def partition(L, left, right):
    """Partitions L[left:right] into L[left:middle], L[middle], and L[middle+1:right]
    according to the last element L[right-1] as pivot."""
    pivot_i = right-1
    pivot = L[pivot_i]
    right -= 1
    i = left
    while i < right:
        print(L[left:i], L[i], L[i+1:right], L[right:pivot_i], L[pivot_i])
        if L[i] < pivot:
            i += 1
        else:
            L[i], L[right-1] = L[right-1], L[i]
            i += 0
            right -= 1
    L[i], L[pivot_i] = L[pivot_i], L[i]
    return i

In [3]:
L = [4, 65, 2, -31, -1, 99, 83, 782, 0]
partition(L, 0, len(L))

[] 4 [65, 2, -31, -1, 99, 83, 782] [] 0
[] 782 [65, 2, -31, -1, 99, 83] [4] 0
[] 83 [65, 2, -31, -1, 99] [782, 4] 0
[] 99 [65, 2, -31, -1] [83, 782, 4] 0
[] -1 [65, 2, -31] [99, 83, 782, 4] 0
[-1] 65 [2, -31] [99, 83, 782, 4] 0
[-1] -31 [2] [65, 99, 83, 782, 4] 0
[-1, -31] 2 [] [65, 99, 83, 782, 4] 0


2

In [4]:
L = [1, 4, 7, 3]
quicksort(L)
assert L == [1, 3, 4, 7]

[] 1 [4, 7] [] 3
[1] 4 [7] [] 3
[1] 7 [] [4] 3
[] 4 [] [] 7


In [5]:
L = [23, 3, 45, 7, 6, 11, 14, 12]
quicksort(L)
assert L == [3, 6, 7, 11, 12, 14, 23, 45]

[] 23 [3, 45, 7, 6, 11, 14] [] 12
[] 14 [3, 45, 7, 6, 11] [23] 12
[] 11 [3, 45, 7, 6] [14, 23] 12
[11] 3 [45, 7, 6] [14, 23] 12
[11, 3] 45 [7, 6] [14, 23] 12
[11, 3] 6 [7] [45, 14, 23] 12
[11, 3, 6] 7 [] [45, 14, 23] 12
[] 11 [3, 6] [] 7
[] 6 [3] [11] 7
[6] 3 [] [11] 7
[] 6 [] [] 3
[] 14 [23] [] 45
[14] 23 [] [] 45
[] 14 [] [] 23
